# 3. Evolve-Filter, steady state
* Completed Steady State, need to make convergence plots
* $\delta = 1/nx$
* $a = a_D$
* Using different N

In [7]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])
# Exact Solution
c = 16.0 #*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

ue = sym.simplify(iliescu)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = 0 #sym.diff(ue,t) Remove time dependence for now
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

In [8]:
# %load main-EFR.py

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

In [12]:
# filter solving

def deconv_filter(u_, delta, Q, boundary):
    u_tilde = TrialFunction(Q)
    v = TestFunction(Q)

    F_Hfilter = v*u_tilde*dx + delta*delta*dot(grad(v), grad(u_tilde))*dx - v*u_*dx 

    a_Hfilter = lhs(F_Hfilter)
    L_Hfilter = rhs(F_Hfilter)
    
    bc = DirichletBC(Q, u_, boundary)
    
    A_Hfilter = assemble(a_Hfilter)
    bc.apply(A_Hfilter)

    b_Hfilter = assemble(L_Hfilter)
    bc.apply(b_Hfilter)

    solver = LUSolver(A_Hfilter)
    u_tilde = Function(Q)
    solver.solve(u_tilde.vector(), b_Hfilter)
    return u_tilde

In [26]:
def evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, velocity):
    degree = 2
    folder = 'EF_N3/'

    
    u_exact = Expression(u_code, degree = degree)  
    f = Expression(f_code, degree = degree)
    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree)
    def boundary(x, on_boundary):
        return on_boundary
    
# ------------ STEP 1: Solve on coarse grid ----------------------#

    bc = DirichletBC(Q, u_D, boundary)

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n = Function(Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Step 1 Solve on Coarse Grid
    b = assemble(L)
    bc.apply(b)
    solve(A1, u_.vector(), b, 'gmres')
    
#     errors = compute_errors(u_exact, u_, 0, mesh)
#     print(errors)

    out_file_uexact = File(folder+"evolve_steady_u_exact_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_uexact << ue
    out_file_ubar = File(folder+str(mu)+"evolve_steady_u_"+str(nx)+".pvd") 
    out_file_ubar << u_

    
# ------------ STEP 2: Helmholtz filter ----------------------#


#     # N = 0
#     u_tilde0 = deconv_filter(u_, delta, Q, boundary)
#     DF = Expression('u_tilde0', degree = degree, u_tilde0 = u_tilde0)

#     # N = 1
#     u_tilde0 = deconv_filter(u_, delta, Q, boundary)
#     u_tilde1 = deconv_filter(u_tilde0, delta, Q, boundary)
#     DF = Expression('2.0*u_tilde0 - u_tilde1', degree = degree, u_tilde0 = u_tilde0, u_tilde1 = u_tilde1)

#     # N = 2
#     u_tilde1 = deconv_filter(u_, delta, Q, boundary)
#     u_tilde2 = deconv_filter(u_tilde1, delta, Q, boundary)
#     u_tilde3 = deconv_filter(u_tilde2, delta, Q, boundary)
#     DF = Expression('3.0*u_tilde1 - 3.0*u_tilde2 + u_tilde3', degree = degree, u_tilde1 = u_tilde1, u_tilde2 = u_tilde2, u_tilde3 = u_tilde3)

    # N = 3
    u_tilde1 = deconv_filter(u_, delta, Q, boundary)
    u_tilde2 = deconv_filter(u_tilde1, delta, Q, boundary)
    u_tilde3 = deconv_filter(u_tilde2, delta, Q, boundary)
    u_tilde4 = deconv_filter(u_tilde3, delta, Q, boundary)
    DF = Expression('3.0*u_tilde1 - 3.0*u_tilde2 + u_tilde3 + u_tilde1 - 3.0*u_tilde2 + 3.0*u_tilde3 - u_tilde4', degree = degree, u_tilde1 = u_tilde1, u_tilde2 = u_tilde2, u_tilde3 = u_tilde3, u_tilde4 = u_tilde4)

    # Compute the indicator function N = 0
    indicator = Expression('sqrt((a-b)*(a-b))', degree = 2, a = u_, b = DF)
    indicator = interpolate(indicator, Q)
    max_ind = np.amax(indicator.vector().get_local())

    if max_ind < 1:
        max_ind = 1.0

    indicator = Expression('a/b', degree = 2, a = indicator, b = max_ind)
    indicator = interpolate(indicator, Q)

    out_file_ind = File(folder+str(mu)+"EF_aD"+str(nx)+".pvd")
    out_file_ind << indicator

    # Apply the filter
    u_bar = TrialFunction(Q)
    F_filter = v*u_bar*dx + delta*delta*dot(grad(v), indicator*grad(u_bar))*dx - v*u_*dx 

    a_filter = lhs(F_filter)
    L_filter = rhs(F_filter)

    A_filter = assemble(a_filter)
    bc.apply(A_filter)

    b_filter = assemble(L_filter)
    bc.apply(b_filter)

    solver = LUSolver(A_filter)
    u_bar = Function(Q)
    solver.solve(u_bar.vector(), b_filter)
    
    out_file_ufilter = File(folder+"delta"+filename+"_filter_u_"+str(nx)+".pvd") 
    out_file_ufilter << u_bar

    
    L2, H1 = compute_errors(u_exact, u_bar, t, mesh)
    maxval, minval = compute_extrema(u_bar, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)


## $ N = 0 $

25 , 0.281849030719 , 15.6615596083 , 1.58232307758 , -0.77453060737
50 , 0.174892605469 , 12.864296479 , 1.48156398067 , -0.518960941244
100 , 0.029634489156 , 7.49899879463 , 1.07818906133 , -0.104262476451
200 , 0.0051976175589 , 3.32468881229 , 1.00559701449 , -0.0300435478052
400 , 0.000418226657031 , 0.643213010063 , 0.992008662377 , -0.000933532363759

## $ N = 1 $
25 , 0.287233502981 , 16.0204889199 , 1.6158368443 , -0.809097878514
50 , 0.175161958643 , 12.8447116586 , 1.49293255105 , -0.536161548846
100 , 0.0299615900264 , 7.71058142427 , 1.08146984486 , -0.109884189636
200 , 0.00520813015613 , 3.40013509262 , 1.00575814501 , -0.0308328352103
400 , 0.000299064877717 , 0.528557170247 , 0.992008870185 , -0.000934400407316

# $ N = 2 $

25 , 0.290681547637 , 16.2660732197 , 1.64387910452 , -0.848598139854
50 , 0.17516303274 , 12.8261324184 , 1.497922101 , -0.534840344007
100 , 0.0301550694171 , 7.84613735063 , 1.08387561473 , -0.1127414027
200 , 0.00521131970991 , 3.43707655396 , 1.00582721641 , -0.0311979550416
400 , 0.000257595209454 , 0.492349863352 , 0.99200887985 , -0.000935200927092

# $ N = 3 $
25 , 0.292534761633 , 16.4051086018 , 1.6680810514 , -0.870443091387
50 , 0.175250235455 , 12.8425344419 , 1.5073793708 , -0.534216168652
100 , 0.0303119710551 , 7.95689306981 , 1.08622686414 , -0.11469786005
200 , 0.00521732741324 , 3.46673866426 , 1.00589350893 , -0.0314428632787
400 , 0.000240362496861 , 0.481830499745 , 0.992008888805 , -0.000935960840084


## $ N = 0 $ with new function
* confirmed that new function does not change error values.

25 , 0.281849030719 , 15.6615596083 , 1.58232307758 , -0.77453060737

50 , 0.174892605469 , 12.864296479 , 1.48156398067 , -0.518960941244

100 , 0.029634489156 , 7.49899879463 , 1.07818906133 , -0.104262476451

200 , 0.0051976175589 , 3.32468881229 , 1.00559701449 , -0.0300435478052

400 , 0.000418226657031 , 0.643213010063 , 0.992008662377 , -0.000933532363759

In [27]:
filename = 'varies'
for nx in [25, 50, 100, 200, 400]:
    delta = 1.0/nx
    evolve_filter(filename, delta, nx, u_code, f_code, sigma, mu, b)

--- Instant: compiling ---


25 , 0.292534761633 , 16.4051086018 , 1.6680810514 , -0.870443091387
50 , 0.175250235455 , 12.8425344419 , 1.5073793708 , -0.534216168652
100 , 0.0303119710551 , 7.95689306981 , 1.08622686414 , -0.11469786005
200 , 0.00521732741324 , 3.46673866426 , 1.00589350893 , -0.0314428632787
400 , 0.000240362496861 , 0.481830499745 , 0.992008888805 , -0.000935960840084
